In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from Bio import Phylo
from Bio import SeqIO

In [ ]:
# Read the newick tree
tree = Phylo.read("tree.nwk", "newick")
# Reset plotting font size and figure size
mpl.rcParams.update({'font.size': 25})
plt.rcParams['figure.figsize'] = [35, 60]
# Draw
Phylo.draw(tree)

In [ ]:
# Load the unaligned, raw sequence file
raw_seq = []
for seq_rec in SeqIO.parse("seqs.fa", "fasta"):
    raw_seq.append(seq_rec)

In [ ]:
def specimen(*args):
    # This function picks representative specimens from the tree
    seq_to_show = []
    for arg in args:
        for seq in raw_seq:
            if seq.id == arg:
                seq_to_show.append(str(seq.seq))
                break
    return seq_to_show

specimen("rh.61", "rh.58", "hu.39", "bb.1", "cy.2")

### Cluster 0
Specimen #0 is named rh.61. The most common match for its cluster is the capsid protein VP1 gene in Adeno-associated virus 
isolate. The top three matches are isolate rh.61, rh.64R1, and rh.46. The mathces are the same gene. There is one match to
Arthrobacter sp.LS16 complete genome which is different from most others.

### Cluster 1
Specimen #1 is named rh.58. The most common match for its cluster is the capsid protein VP1 gene in Adeno-associated virus 
isolate. The top three matches are isolate rh.58, rh.64R1, and rh.46. The mathces are the same gene. There is one match to
Arthrobacter sp.LS16 complete genome which is different from most others.

### Cluster 2
Specimen #2 is named hu.39. The most common match for its cluster is the capsid protein VP1 gene in Adeno-associated virus 
isolate. The top three matches are isolate hu.39, rh.64R1, and rh.46. The mathces are the same gene. There is one match to
Arthrobacter sp.LS16 complete genome which is different from most others.

### Cluster 3
Specimen #3 is named bb.1. The most common match for its cluster is the capsid protein VP1 gene in non-human primate
Adeno-associated virus isolate. The top three matches are isolate bb.1, bb.2, and rh.10. The mathces are the same
gene. There is one match to Arthrobacter sp.LS16 complete genome which is different from most others.

### Cluster 4
Specimen #4 is named cy.2. The most common match for its cluster is the capsid protein VP1 gene in Adeno-associated virus
isolate. The top three matches are isolate cy.2, AAV 7, and Anc113. The mathces are the same gene. There is one match to 
Arthrobacter sp.LS16 complete genome which is different from most others.

**Q: Why might we not trust the annotations for sequences that come up in our BLAST?**
<br>
A: Although the specimen of interest is a type of virus which does not have complex mRNA modification mechanism, the same gene can be expressed as different proteins in different eukaryotic organisms. The same gene sequence does not necessarily result in the same protein product. In addition, proteins often have various domains. The function of one domain does not necessarily disctate the function of the overall protein. Conversely, the function of a protein does not disclose the biochemical properties of its sub-domains.
<br>
We also have no control over the quality of the annotation database. Although NCBI is trustworthy, 

In [ ]:
# Clusters determined by eyeballing the tree graph
cluster0 = ["rh.61"]
cluster1 = ["rh.58"]
cluster2 = ["hu.39", "rh.64", "rh.57", "rh.51", "rh.49", "rh.53", "rh.50", "rh.52"]
cluster3 = ["bb.1", "pi.2", "pi.3", "pi.1", "rh.10", "bb.2", "hu.17", "hu.6", "rh.2", "rh.40", "hu.67", "hu.37", "hu.40",
           "hu.66", "hu.41", "rh.38", "hu.42"]
cluster4 = ["cy.2", "cy.3", "cy.6", "cy.4", "cy.5", "rh.13", "rh.35", "rh.36", "rh.37", "rh.54", "rh.55", "rh.48", "rh.62",
           "hu.43", "hu.48", "hu.44", "hu.46", "hu.14", "hu.31", "hu.32", "rh.43"]

In [ ]:
# Load the aligned sequence file
aligned_seq = []
for seq_rec in SeqIO.parse("seqs.aligned.fa","fasta"):
    aligned_seq.append(seq_rec)

In [ ]:
def clust_grouper(*args):
    # This function takes in a list of specimen ids and returns a list of the corresponding Bio.Seq objects from aligned_seq
    ls = []
    for arg in args:
        for obj in aligned_seq:
            if obj.id == arg:
                ls.append(obj)
                break
    return ls

cl0 = clust_grouper(*cluster0)
cl1 = clust_grouper(*cluster1)
cl2 = clust_grouper(*cluster2)
cl3 = clust_grouper(*cluster3)
cl4 = clust_grouper(*cluster4)

In [ ]:
def ATGC_by_cluster(clust):
    # ATGC_by_cluster() returns a tuple value which is composed of two lists--a list of %AT and a list of %CG. 
    # The individual value within either list is calculated based on the GC or AT content at a particular position 
    # across every sequence in a user-defined cluster.
    # The parameter, clust, is a list of all the Bio.Seq objects within a user-defined cluster.

    # Initialization
    length = len(clust[0].seq)
    percent_AT = [0]*length
    percent_GC = [0]*length
    # The outer loop is incremented based on position
    for pos in range(length):
        GC_count = 0
        AT_count = 0
        #The inner loop looks through the particular position across every sequence in the cluster
        for obj in clust:
            if obj.seq[pos] == "C" or obj.seq[pos] == "G":
                GC_count += 1
            elif obj.seq[pos] == "A" or obj.seq[pos] == "T":
                AT_count += 1
        # This if statement takes care of the occurrence of alignment gaps
        if GC_count + AT_count == 0:
            percent_AT[pos] = 0
            percent_GC[pos] = 0
        # Otherwise, there is at least one identifiable nucleotide at this position. Proceed to calculate the percent normally.
        else:
            percent_AT[pos] = AT_count/(AT_count + GC_count)
            percent_GC[pos] = GC_count/(AT_count + GC_count)
    return percent_AT, percent_GC

clust0_AT, clust0_GC = ATGC_by_cluster(cl0)
clust1_AT, clust1_GC = ATGC_by_cluster(cl1)
clust2_AT, clust2_GC = ATGC_by_cluster(cl2)
clust3_AT, clust3_GC = ATGC_by_cluster(cl3)
clust4_AT, clust4_GC = ATGC_by_cluster(cl4)

In [ ]:
# Reset the plt output figure size
plt.rcParams['figure.figsize'] = [10, 10]
p1 = plt.bar(x = range(len(clust0_GC)), height = clust0_GC, color = "cyan")
p2 = plt.bar(x = range(len(clust0_AT)), height = clust0_AT, bottom = clust0_GC, color = "magenta")
px = plt.xlabel("Position in the cluster genome")
py = plt.ylabel("GC (cyan) or AT (magenta) content (%)")
# Zoom in on position 0 to 50; to see the entire length of the cluster sequences, comment out the line below
# p3 = plt.axis([-0.5, 50.5, 0, 1])

In [ ]:
p1 = plt.bar(x = range(len(clust1_GC)), height = clust1_GC, color = "cyan")
p2 = plt.bar(x = range(len(clust1_AT)), height = clust1_AT, bottom = clust1_GC, color = "magenta")
px = plt.xlabel("Position in the cluster genome")
py = plt.ylabel("GC (cyan) or AT (magenta) content (%)")
# Zoom in on position 0 to 50; to see the entire length of the cluster sequences, comment out the line below
# p3 = plt.axis([-0.5, 50.5, 0, 1])

In [ ]:
p1 = plt.bar(x = range(len(clust2_GC)), height = clust2_GC, color = "cyan")
p2 = plt.bar(x = range(len(clust2_AT)), height = clust2_AT, bottom = clust2_GC, color = "magenta")
px = plt.xlabel("Position in the cluster genome")
py = plt.ylabel("GC (cyan) or AT (magenta) content (%)")
# Zoom in on position 0 to 50; to see the entire length of the cluster sequences, comment out the line below
# p3 = plt.axis([-0.5, 50.5, 0, 1])

In [ ]:
p1 = plt.bar(x = range(len(clust3_GC)), height = clust3_GC, color = "cyan")
p2 = plt.bar(x = range(len(clust3_AT)), height = clust3_AT, bottom = clust3_GC, color = "magenta")
px = plt.xlabel("Position in the cluster genome")
py = plt.ylabel("GC (cyan) or AT (magenta) content (%)")
# Zoom in on position 0 to 50; to see the entire length of the cluster sequences, comment out the line below
# p3 = plt.axis([-0.5, 50.5, 0, 1])

In [ ]:
p1 = plt.bar(x = range(len(clust4_GC)), height = clust4_GC, color = "cyan")
p2 = plt.bar(x = range(len(clust4_AT)), height = clust4_AT, bottom = clust4_GC, color = "magenta")
px = plt.xlabel("Position in the cluster genome")
py = plt.ylabel("GC (cyan) or AT (magenta) content (%)")
# Zoom in on position 0 to 50; to see the entire length of the cluster sequences, comment out the line below
# p3 = plt.axis([-0.5, 50.5, 0, 1])

In [ ]:
def cluster_raw_seq_grouper(*args):
    # This function takes in a list of specimen ids and returns a list of Bio.Seq objects.
    ls = []
    for arg in args:
        for obj in raw_seq:
            if obj.id == arg:
                ls.append(obj)
                break
    return ls

# Output the files of each visually determined cluster
cl0 = cluster_raw_seq_grouper(*cluster0)
cl1 = cluster_raw_seq_grouper(*cluster1)
cl2 = cluster_raw_seq_grouper(*cluster2)
cl3 = cluster_raw_seq_grouper(*cluster3)
cl4 = cluster_raw_seq_grouper(*cluster4)

In [ ]:
def cluster_length(clust):
    # This function takes in a lsit of Bio.Seq objects and returns a list of integers that denotes the length of each 
    # sequence in the list.
    ls_len = []
    ptr = 0
    for seq in clust:
        ls_len.append(len(seq))
    return ls_len

clust0_len = cluster_length(cl0)
clust1_len = cluster_length(cl1)
clust2_len = cluster_length(cl2)
clust3_len = cluster_length(cl3)
clust4_len = cluster_length(cl4)

In [ ]:
# Plot the box plot for cluter sequence length
plt.rcParams['figure.figsize'] = [10, 8]
b1 = plt.boxplot([clust0_len, clust1_len, clust2_len, clust3_len, clust4_len], 
                 labels = ["clust0", "clust1", "clust2", "clust3", "clust4"])